In [1]:
import pandas as pd,warnings,numpy as np,matplotlib.pyplot as plt,seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,plot_confusion_matrix,r2_score,mean_absolute_percentage_error
from sklearn.tree import plot_tree,export_text
from sklearn.datasets import load_boston,load_iris
warnings.filterwarnings('ignore')

In [2]:
def Evaluate_Regressor(model,X_test,y_test):
    y_pred = model.predict(X_test)
    errors = abs(y_pred - y_test)
    mape = 100 * np.mean(errors/y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

In [3]:
def R2_score(model,X,y,name_r_score):
    y_pred = model.predict(X)
    print('{}: {:.2%}'.format(name_r_score,r2_score(y,y_pred)))

# PARAMETERS

In [4]:
n_estimators = [10, 50, 100, 500]
learning_rate = [0.0001, 0.001, 0.01, 0.1, 1.0]
loss = ['linear','square','exponential']
ada_params = {'n_estimators': n_estimators,'learning_rate':learning_rate,'loss':loss}

# DATA

In [5]:
boston_df = load_boston()
X = pd.DataFrame(boston_df.data,columns=boston_df.feature_names)
y = boston_df.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# BASE  MODEL

In [6]:
base_model = AdaBoostRegressor()
base_model.fit(X_train,y_train)

AdaBoostRegressor()

# RandomizedSearchCV

In [7]:
def Hypertuning_rscv(model,params,n_iter,cv,X_train,y_train):
    random = RandomizedSearchCV(estimator=model, param_distributions=params, n_jobs=-1, n_iter=n_iter, cv=cv,verbose=2, random_state=42)
    random.fit(X_train,y_train)
    best_params = random.best_params_
    best_score = random.best_score_
    best_estimator = random.best_estimator_
    return best_params,best_score,best_estimator

In [8]:
best_params, best_score, best_estimator_random = Hypertuning_rscv(AdaBoostRegressor(),ada_params,10,10,X_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


# GridSearchCV

In [9]:
def Hypertuning_gscv(model,params,cv,X_train,y_train):
    grid = GridSearchCV(estimator = model, param_grid = params, cv=cv, n_jobs=-1, verbose=2)
    grid.fit(X_train,y_train)
    best_score = grid.best_score_
    best_params = grid.best_params_
    best_estimator = grid.best_estimator_
    return best_params,best_score,best_estimator

In [10]:
best_params, best_score, best_estimator_grid = Hypertuning_gscv(AdaBoostRegressor(),ada_params,3,X_train,y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


# Evaluation

R2 SCORE

In [11]:
R2_score(base_model,X_train,y_train,'Train R2 score Base Model')
R2_score(base_model,X_test,y_test,'Test R2 score Base Model')
print('')
R2_score(best_estimator_random,X_train,y_train,'Train R2 score Randomimzed')
R2_score(best_estimator_random,X_test,y_test,'Test R2 score Randomized')
print('')
R2_score(best_estimator_grid,X_train,y_train,'Train R2 score Grid')
R2_score(best_estimator_grid,X_test,y_test,'Test R2 scorey Grid')

Train R2 score Base Model: 90.73%
Test R2 score Base Model: 80.41%

Train R2 score Randomimzed: 91.35%
Test R2 score Randomized: 79.82%

Train R2 score Grid: 91.48%
Test R2 scorey Grid: 80.19%


MEAN ERROR & MAPE

In [12]:
base_accuracy = Evaluate_Regressor(base_model,X_test,y_test)

Model Performance
Average Error: 2.7564 degrees.
Accuracy = 84.76%.


In [13]:
random_accuracy_regressor = Evaluate_Regressor(best_estimator_random,X_test,y_test)

Model Performance
Average Error: 2.7264 degrees.
Accuracy = 85.48%.


In [14]:
print('Improvement of {:0.2f}%.'.format(100*(random_accuracy_regressor-base_accuracy)/base_accuracy))

Improvement of 0.86%.


In [16]:
grid_accuracy_regressor = Evaluate_Regressor(best_estimator_grid,X_test,y_test)

Model Performance
Average Error: 2.7708 degrees.
Accuracy = 84.81%.


In [17]:
print('Improvement of {:0.2f}%.'.format(100*(grid_accuracy_regressor-base_accuracy)/base_accuracy))

Improvement of 0.06%.
